In [1]:
from transformers.models.whisper.modeling_whisper import WhisperEncoder
from transformers import WhisperProcessor
from torchaudio import load
from torchaudio.functional import resample
import torch

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
sample_wav_path = '/Users/markjos/projects/malachor5/data/tira-ac/wav/Audacity Recording 181[383000.0-385900.0].wav'
sample_wav_path = '/Users/markjos/projects/malachor5/data/tira-ac/wav/Audacity Recording 204[529700.0-530800.0].wav'
sample_wav, sr = load(sample_wav_path)
sample_wav.shape, sr

(torch.Size([1, 48510]), 44100)

In [55]:
sample_wav = resample(sample_wav, sr, 16_000)[0]
sample_wav.shape

torch.Size([17600])

In [22]:
proc=WhisperProcessor.from_pretrained("openai/whisper-tiny")
proc

WhisperProcessor:
- feature_extractor: WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: WhisperTokenizer(name_or_path='openai/whisper-tiny', vocab_size=50258, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<|hi|>', '<|fi|>', '<|vi|>', '<|he|>', '<|uk|>', 

In [23]:
model=WhisperEncoder.from_pretrained("openai/whisper-tiny")

Some weights of WhisperEncoder were not initialized from the model checkpoint at openai/whisper-tiny and are newly initialized: ['model.conv1.bias', 'model.conv1.weight', 'model.conv2.bias', 'model.conv2.weight', 'model.embed_positions.weight', 'model.layer_norm.bias', 'model.layer_norm.weight', 'model.layers.0.fc1.bias', 'model.layers.0.fc1.weight', 'model.layers.0.fc2.bias', 'model.layers.0.fc2.weight', 'model.layers.0.final_layer_norm.bias', 'model.layers.0.final_layer_norm.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.out_proj.bias', 'model.layers.0.self_attn.out_proj.weight', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.0.self_attn_layer_norm.bias', 'model.layers.0.self_attn_layer_norm.weight', 'model.layers.1.fc1.bias', 'model.layers.1.fc1.weight', 'model.layers.1.fc2.bias', 'model.layers.1.fc2.weight', 'model.layers.1

In [58]:
input_dict = proc.feature_extractor(sample_wav, sampling_rate=16_000, return_tensors='pt')
input_dict

{'input_features': tensor([[[-0.7875, -0.7875, -0.7875,  ..., -0.7875, -0.7875, -0.7875],
         [-0.7875, -0.7875, -0.7875,  ..., -0.7875, -0.7875, -0.7875],
         [-0.7875, -0.7875, -0.7875,  ..., -0.7875, -0.7875, -0.7875],
         ...,
         [-0.7875, -0.7875, -0.7875,  ..., -0.7875, -0.7875, -0.7875],
         [-0.7875, -0.7875, -0.7875,  ..., -0.7875, -0.7875, -0.7875],
         [-0.7875, -0.7875, -0.7875,  ..., -0.7875, -0.7875, -0.7875]]])}

In [47]:
input_dict.keys()

dict_keys(['input_features'])

In [60]:
input_dict['input_features'].shape

torch.Size([1, 80, 3000])

In [59]:
with torch.no_grad():
    output = model(input_dict['input_features'])
output, output.keys(), output['last_hidden_state'].shape

(BaseModelOutput(last_hidden_state=tensor([[[-1.1402, -0.7276, -0.4304,  ...,  0.1594,  1.2768,  1.4110],
          [-0.0346,  0.7456,  1.0651,  ...,  0.0383,  1.1219,  1.1018],
          [-0.0283,  1.2714,  1.5932,  ..., -0.0814,  0.7749,  0.8872],
          ...,
          [ 0.2692, -0.0753,  1.3027,  ...,  0.3314,  0.2429,  0.2534],
          [-0.3495,  0.8758,  0.3711,  ...,  0.4158,  0.3248,  0.2740],
          [-1.5565,  0.7671, -0.5956,  ...,  0.4801,  0.4508,  0.4085]]]), hidden_states=None, attentions=None),
 odict_keys(['last_hidden_state']),
 torch.Size([1, 1500, 384]))

In [68]:
hidden_state = output['last_hidden_state']
hidden_state[:,-1]

tensor([[-1.5565e+00,  7.6706e-01, -5.9564e-01,  1.2076e-01, -6.2636e-01,
         -1.2003e-01, -9.4798e-01,  8.7108e-01,  1.9926e+00, -6.3919e-01,
          1.0913e+00,  8.8971e-01, -2.2728e+00,  5.6017e-01,  1.4054e-01,
         -1.9608e+00,  5.6128e-01,  1.0878e+00,  1.4876e+00,  3.4111e-01,
         -5.6921e-01, -1.6409e+00, -1.1856e+00, -9.2948e-01, -3.8147e-01,
          1.0827e+00,  5.8549e-01, -1.9555e+00, -1.9496e+00, -7.2708e-01,
          8.4288e-01,  3.5045e-01,  2.8577e-01, -3.6884e-01,  6.1430e-01,
          5.8921e-01,  3.9006e-02, -7.7440e-02,  1.1263e+00,  1.2544e+00,
         -1.4921e+00, -2.6051e-01, -7.5920e-02, -2.9420e-01, -1.2440e+00,
          1.2133e+00, -1.6426e+00, -7.1546e-01, -9.7428e-01,  4.6478e-01,
          8.0324e-01,  1.1481e+00,  8.1543e-01,  3.0179e-01, -1.2911e+00,
         -1.9412e+00,  3.4232e-01,  1.1337e+00, -1.0084e+00, -8.8159e-01,
          9.1499e-01, -1.1249e+00, -2.4260e-01, -1.1726e-01, -1.6692e+00,
          3.8879e-01, -1.4306e+00,  4.

In [5]:
import os
from collections import OrderedDict

import datasets

logger = datasets.logging.get_logger(__name__)


""" FLEURS Dataset"""

_FLEURS_LANG_TO_ID = OrderedDict([("Afrikaans", "af"), ("Amharic", "am"), ("Arabic", "ar"), ("Armenian", "hy"), ("Assamese", "as"), ("Asturian", "ast"), ("Azerbaijani", "az"), ("Belarusian", "be"), ("Bengali", "bn"), ("Bosnian", "bs"), ("Bulgarian", "bg"), ("Burmese", "my"), ("Catalan", "ca"), ("Cebuano", "ceb"), ("Mandarin Chinese", "cmn_hans"), ("Cantonese Chinese", "yue_hant"), ("Croatian", "hr"), ("Czech", "cs"), ("Danish", "da"), ("Dutch", "nl"), ("English", "en"), ("Estonian", "et"), ("Filipino", "fil"), ("Finnish", "fi"), ("French", "fr"), ("Fula", "ff"), ("Galician", "gl"), ("Ganda", "lg"), ("Georgian", "ka"), ("German", "de"), ("Greek", "el"), ("Gujarati", "gu"), ("Hausa", "ha"), ("Hebrew", "he"), ("Hindi", "hi"), ("Hungarian", "hu"), ("Icelandic", "is"), ("Igbo", "ig"), ("Indonesian", "id"), ("Irish", "ga"), ("Italian", "it"), ("Japanese", "ja"), ("Javanese", "jv"), ("Kabuverdianu", "kea"), ("Kamba", "kam"), ("Kannada", "kn"), ("Kazakh", "kk"), ("Khmer", "km"), ("Korean", "ko"), ("Kyrgyz", "ky"), ("Lao", "lo"), ("Latvian", "lv"), ("Lingala", "ln"), ("Lithuanian", "lt"), ("Luo", "luo"), ("Luxembourgish", "lb"), ("Macedonian", "mk"), ("Malay", "ms"), ("Malayalam", "ml"), ("Maltese", "mt"), ("Maori", "mi"), ("Marathi", "mr"), ("Mongolian", "mn"), ("Nepali", "ne"), ("Northern-Sotho", "nso"), ("Norwegian", "nb"), ("Nyanja", "ny"), ("Occitan", "oc"), ("Oriya", "or"), ("Oromo", "om"), ("Pashto", "ps"), ("Persian", "fa"), ("Polish", "pl"), ("Portuguese", "pt"), ("Punjabi", "pa"), ("Romanian", "ro"), ("Russian", "ru"), ("Serbian", "sr"), ("Shona", "sn"), ("Sindhi", "sd"), ("Slovak", "sk"), ("Slovenian", "sl"), ("Somali", "so"), ("Sorani-Kurdish", "ckb"), ("Spanish", "es"), ("Swahili", "sw"), ("Swedish", "sv"), ("Tajik", "tg"), ("Tamil", "ta"), ("Telugu", "te"), ("Thai", "th"), ("Turkish", "tr"), ("Ukrainian", "uk"), ("Umbundu", "umb"), ("Urdu", "ur"), ("Uzbek", "uz"), ("Vietnamese", "vi"), ("Welsh", "cy"), ("Wolof", "wo"), ("Xhosa", "xh"), ("Yoruba", "yo"), ("Zulu", "zu")])
_FLEURS_LANG_SHORT_TO_LONG = {v: k for k, v in _FLEURS_LANG_TO_ID.items()}


_FLEURS_LANG = sorted(["af_za", "am_et", "ar_eg", "as_in", "ast_es", "az_az", "be_by", "bn_in", "bs_ba", "ca_es", "ceb_ph", "cmn_hans_cn", "yue_hant_hk", "cs_cz", "cy_gb", "da_dk", "de_de", "el_gr", "en_us", "es_419", "et_ee", "fa_ir", "ff_sn", "fi_fi", "fil_ph", "fr_fr", "ga_ie", "gl_es", "gu_in", "ha_ng", "he_il", "hi_in", "hr_hr", "hu_hu", "hy_am", "id_id", "ig_ng", "is_is", "it_it", "ja_jp", "jv_id", "ka_ge", "kam_ke", "kea_cv", "kk_kz", "km_kh", "kn_in", "ko_kr", "ckb_iq", "ky_kg", "lb_lu", "lg_ug", "ln_cd", "lo_la", "lt_lt", "luo_ke", "lv_lv", "mi_nz", "mk_mk", "ml_in", "mn_mn", "mr_in", "ms_my", "mt_mt", "my_mm", "nb_no", "ne_np", "nl_nl", "nso_za", "ny_mw", "oc_fr", "om_et", "or_in", "pa_in", "pl_pl", "ps_af", "pt_br", "ro_ro", "ru_ru", "bg_bg", "sd_in", "sk_sk", "sl_si", "sn_zw", "so_so", "sr_rs", "sv_se", "sw_ke", "ta_in", "te_in", "tg_tj", "th_th", "tr_tr", "uk_ua", "umb_ao", "ur_pk", "uz_uz", "vi_vn", "wo_sn", "xh_za", "yo_ng", "zu_za"])
_FLEURS_LONG_TO_LANG = {_FLEURS_LANG_SHORT_TO_LONG["_".join(k.split("_")[:-1]) or k]: k for k in _FLEURS_LANG}
_FLEURS_LANG_TO_LONG = {v: k for k, v in _FLEURS_LONG_TO_LANG.items()}

_FLEURS_GROUP_TO_LONG = OrderedDict({
    "western_european_we": ["Asturian", "Bosnian", "Catalan", "Croatian", "Danish", "Dutch", "English", "Finnish", "French", "Galician", "German", "Greek", "Hungarian", "Icelandic", "Irish", "Italian", "Kabuverdianu", "Luxembourgish", "Maltese", "Norwegian", "Occitan", "Portuguese", "Spanish", "Swedish", "Welsh"],
    "eastern_european_ee": ["Armenian", "Belarusian", "Bulgarian", "Czech", "Estonian", "Georgian", "Latvian", "Lithuanian", "Macedonian", "Polish", "Romanian", "Russian", "Serbian", "Slovak", "Slovenian", "Ukrainian"],
    "central_asia_middle_north_african_cmn": ["Arabic", "Azerbaijani", "Hebrew", "Kazakh", "Kyrgyz", "Mongolian", "Pashto", "Persian", "Sorani-Kurdish", "Tajik", "Turkish", "Uzbek"],
    "sub_saharan_african_ssa": ["Afrikaans", "Amharic", "Fula", "Ganda", "Hausa", "Igbo", "Kamba", "Lingala", "Luo", "Northern-Sotho", "Nyanja", "Oromo", "Shona", "Somali", "Swahili", "Umbundu", "Wolof", "Xhosa", "Yoruba", "Zulu"],
    "south_asian_sa": ["Assamese", "Bengali", "Gujarati", "Hindi", "Kannada", "Malayalam", "Marathi", "Nepali", "Oriya", "Punjabi", "Sindhi", "Tamil", "Telugu", "Urdu"],
    "south_east_asian_sea": ["Burmese", "Cebuano", "Filipino", "Indonesian", "Javanese", "Khmer", "Lao", "Malay", "Maori", "Thai", "Vietnamese"],
    "chinese_japanase_korean_cjk": ["Mandarin Chinese", "Cantonese Chinese", "Japanese", "Korean"],
})
_FLEURS_LONG_TO_GROUP = {a: k for k, v in _FLEURS_GROUP_TO_LONG.items() for a in v}
_FLEURS_LANG_TO_GROUP = {_FLEURS_LONG_TO_LANG[k]: v for k, v in _FLEURS_LONG_TO_GROUP.items()}

_ALL_LANG = _FLEURS_LANG
_ALL_CONFIGS = []

In [6]:
from datasets import load_dataset
from transformers.models.whisper.tokenization_whisper import LANGUAGES, TO_LANGUAGE_CODE

In [6]:
_FLEURS_LONG_TO_LANG

{'Afrikaans': 'af_za',
 'Amharic': 'am_et',
 'Arabic': 'ar_eg',
 'Assamese': 'as_in',
 'Asturian': 'ast_es',
 'Azerbaijani': 'az_az',
 'Belarusian': 'be_by',
 'Bulgarian': 'bg_bg',
 'Bengali': 'bn_in',
 'Bosnian': 'bs_ba',
 'Catalan': 'ca_es',
 'Cebuano': 'ceb_ph',
 'Sorani-Kurdish': 'ckb_iq',
 'Mandarin Chinese': 'cmn_hans_cn',
 'Czech': 'cs_cz',
 'Welsh': 'cy_gb',
 'Danish': 'da_dk',
 'German': 'de_de',
 'Greek': 'el_gr',
 'English': 'en_us',
 'Spanish': 'es_419',
 'Estonian': 'et_ee',
 'Persian': 'fa_ir',
 'Fula': 'ff_sn',
 'Finnish': 'fi_fi',
 'Filipino': 'fil_ph',
 'French': 'fr_fr',
 'Irish': 'ga_ie',
 'Galician': 'gl_es',
 'Gujarati': 'gu_in',
 'Hausa': 'ha_ng',
 'Hebrew': 'he_il',
 'Hindi': 'hi_in',
 'Croatian': 'hr_hr',
 'Hungarian': 'hu_hu',
 'Armenian': 'hy_am',
 'Indonesian': 'id_id',
 'Igbo': 'ig_ng',
 'Icelandic': 'is_is',
 'Italian': 'it_it',
 'Japanese': 'ja_jp',
 'Javanese': 'jv_id',
 'Georgian': 'ka_ge',
 'Kamba': 'kam_ke',
 'Kabuverdianu': 'kea_cv',
 'Kazakh': 'kk_kz

In [7]:
len(_FLEURS_LONG_TO_LANG)

102

In [12]:
# build a dictionary mapping FLEURS language codes to Whisper language names
# start with overlapping items
whisper_to_fleurs = {}
not_in_fleurs = []
fleurs_lower = {k.lower(): v for k, v in _FLEURS_LONG_TO_LANG.items()}
for language in LANGUAGES.values():
    if language in fleurs_lower:
        whisper_to_fleurs[language] = fleurs_lower[language]
    else:
        not_in_fleurs.append(language)
len(whisper_to_fleurs), len(not_in_fleurs)
    

(79, 21)

In [13]:
not_in_fleurs

['chinese',
 'latin',
 'breton',
 'basque',
 'albanian',
 'sinhala',
 'yiddish',
 'faroese',
 'haitian creole',
 'turkmen',
 'nynorsk',
 'sanskrit',
 'myanmar',
 'tibetan',
 'tagalog',
 'malagasy',
 'tatar',
 'hawaiian',
 'bashkir',
 'sundanese',
 'cantonese']

In [16]:
# identify other mappings by hand
whisper_to_fleurs['chinese'] = 'cmn_hans'
whisper_to_fleurs['cantonese'] = 'yue_hant'
whisper_to_fleurs['tagalog'] = 'fil' # different ISO codes ('tl' vs 'fil') but I'm assuming these are the same for now
whisper_to_fleurs['myanmar'] = 'my'

not_in_fleurs.remove('chinese')
not_in_fleurs.remove('cantonese')
not_in_fleurs.remove('tagalog')
not_in_fleurs.remove('myanmar')
len(whisper_to_fleurs), not_in_fleurs

(83,
 ['latin',
  'breton',
  'basque',
  'albanian',
  'sinhala',
  'yiddish',
  'faroese',
  'haitian creole',
  'turkmen',
  'nynorsk',
  'sanskrit',
  'tibetan',
  'malagasy',
  'tatar',
  'hawaiian',
  'bashkir',
  'sundanese'])

In [17]:
whisper_to_fleurs

{'english': 'en_us',
 'german': 'de_de',
 'spanish': 'es_419',
 'russian': 'ru_ru',
 'korean': 'ko_kr',
 'french': 'fr_fr',
 'japanese': 'ja_jp',
 'portuguese': 'pt_br',
 'turkish': 'tr_tr',
 'polish': 'pl_pl',
 'catalan': 'ca_es',
 'dutch': 'nl_nl',
 'arabic': 'ar_eg',
 'swedish': 'sv_se',
 'italian': 'it_it',
 'indonesian': 'id_id',
 'hindi': 'hi_in',
 'finnish': 'fi_fi',
 'vietnamese': 'vi_vn',
 'hebrew': 'he_il',
 'ukrainian': 'uk_ua',
 'greek': 'el_gr',
 'malay': 'ms_my',
 'czech': 'cs_cz',
 'romanian': 'ro_ro',
 'danish': 'da_dk',
 'hungarian': 'hu_hu',
 'tamil': 'ta_in',
 'norwegian': 'nb_no',
 'thai': 'th_th',
 'urdu': 'ur_pk',
 'croatian': 'hr_hr',
 'bulgarian': 'bg_bg',
 'lithuanian': 'lt_lt',
 'maori': 'mi_nz',
 'malayalam': 'ml_in',
 'welsh': 'cy_gb',
 'slovak': 'sk_sk',
 'telugu': 'te_in',
 'persian': 'fa_ir',
 'latvian': 'lv_lv',
 'bengali': 'bn_in',
 'serbian': 'sr_rs',
 'azerbaijani': 'az_az',
 'slovenian': 'sl_si',
 'kannada': 'kn_in',
 'estonian': 'et_ee',
 'macedonia

In [3]:
en = load_dataset("google/fleurs", _FLEURS_LONG_TO_LANG["English"], split='test')
en

Extracting data files: 100%|██████████| 3/3 [00:17<00:00,  5.87s/it]
Generating train split: 2602 examples [00:15, 166.55 examples/s]
Generating validation split: 394 examples [00:02, 186.90 examples/s]
Generating test split: 647 examples [00:03, 180.01 examples/s]


Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 647
})

In [8]:
sp = load_dataset("google/fleurs", _FLEURS_LONG_TO_LANG["Spanish"], split='test')
sp

Extracting data files: 100%|██████████| 3/3 [00:20<00:00,  6.91s/it]
Generating train split: 2796 examples [00:16, 173.23 examples/s]
Generating validation split: 408 examples [00:02, 156.56 examples/s]
Generating test split: 908 examples [00:05, 153.08 examples/s]


In [14]:
sp=sp.sort("id")
en=en.sort("id")

In [17]:
en['transcription'][:5]

['romanticism had a large element of cultural determinism drawn from writers such as goethe fichte and schlegel',
 'romanticism had a large element of cultural determinism drawn from writers such as goethe fichte and schlegel',
 "he did not set a figure for the cuts saying they will be made based on china's economic output",
 "he did not set a figure for the cuts saying they will be made based on china's economic output",
 "alloys are basically a mixture of two or more metals don't forget that there are many elements on the periodic table"]

In [18]:
sp['transcription'][:5]

['el elemento del determinismo cultural se encontraba muy presente en el romanticismo según estudiosos como goether fichte y schlegel',
 'el elemento del determinismo cultural se encontraba muy presente en el romanticismo según estudiosos como goether fichte y schlegel',
 'no mencionó una cifra para los recortes y manifestó que se harán en base a la producción económica de china',
 'no mencionó una cifra para los recortes y manifestó que se harán en base a la producción económica de china',
 'no mencionó una cifra para los recortes y manifestó que se harán en base a la producción económica de china']

In [30]:
sp_audio=sp['audio'][:3]
sp_audio

[{'path': 'test/16910986247957294711.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.0033735 ,
         -0.00327235, -0.00327545]),
  'sampling_rate': 16000},
 {'path': 'test/3745768066060440719.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00207382,
         -0.00193042, -0.00103712]),
  'sampling_rate': 16000},
 {'path': 'test/6253518291639218682.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00151247,
         -0.00042576,  0.0002048 ]),
  'sampling_rate': 16000}]

In [33]:
input_dict=proc([row['array'] for row in sp_audio], sampling_rate=sp_audio[0]['sampling_rate'], return_tensors='pt')
input_dict

{'input_features': tensor([[[-0.6663, -0.6663, -0.6663,  ..., -0.6663, -0.6663, -0.6663],
         [-0.6663, -0.6663, -0.6663,  ..., -0.6663, -0.6663, -0.6663],
         [-0.6663, -0.6663, -0.6663,  ..., -0.6663, -0.6663, -0.6663],
         ...,
         [-0.6663, -0.6663, -0.6663,  ..., -0.6663, -0.6663, -0.6663],
         [-0.6663, -0.6663, -0.6663,  ..., -0.6663, -0.6663, -0.6663],
         [-0.6663, -0.6663, -0.6663,  ..., -0.6663, -0.6663, -0.6663]],

        [[-0.6495, -0.6495, -0.6495,  ..., -0.6495, -0.6495, -0.6495],
         [-0.6495, -0.6495, -0.6495,  ..., -0.6495, -0.6495, -0.6495],
         [-0.6495, -0.6495, -0.6495,  ..., -0.6495, -0.6495, -0.6495],
         ...,
         [-0.6495, -0.6495, -0.6495,  ..., -0.6495, -0.6495, -0.6495],
         [-0.6495, -0.6495, -0.6495,  ..., -0.6495, -0.6495, -0.6495],
         [-0.6495, -0.6495, -0.6495,  ..., -0.6495, -0.6495, -0.6495]],

        [[-0.6536, -0.6536, -0.6536,  ..., -0.6536, -0.6536, -0.6536],
         [-0.6536, -0.6536

In [38]:
hidden_states=model(input_dict['input_features'])['last_hidden_state']
hidden_states.shape


torch.Size([3, 1500, 384])

In [45]:
embeds=torch.mean(hidden_states, 1)
embeds.shape

torch.Size([3, 384])

In [46]:
torch.concat([embeds, embeds], dim=0).shape

torch.Size([6, 384])